In [1]:
import pandas as pd
from ast import literal_eval
from random import choice
import sys
sys.path.append('../src/')
import data_layer

df_wiki = pd.read_csv("../dat/wiki/graph.csv")
df_llm = pd.read_json("../dat/llm/graph.json")
df_llm['deps'] = df_llm['dep_articles']
df_txb = pd.read_json("../dat/textbooks/graph.json").iloc[:,[0,3]]
df_txb.columns=['concept','deps']
df_wiki['deps'] = [[dep[1] for dep in literal_eval(deps)] for deps in (df_wiki.dep_articles)]

In [5]:
MIN_OCCURENCES=5

In [2]:
_, _, df_count = data_layer.read_index_and_wiki_concepts(include_counts=True)


In [12]:


df_count.sort_values('count', ascending=False)

,concept,count
103,Matrix (mathematics),10
184,Triangular matrix,10
108,Matrix multiplication,10
11,Canonical form,9
175,System of linear equations,9
...,...,...
802,Plane of rotation,1
803,Poietic Generator,1
804,Point-biserial correlation coefficient,1
805,Pointwise product,1


In [3]:
n=100
rnd_st=123
concepts = []
deps = []

for df in [df_llm, df_wiki,df_txb]:
    df = pd.merge(df, df_count, on="concept")
    df = df.loc[(df.deps.str.len() > 0) & (df['count']>=MIN_OCCURENCES)]
    samp = df.sample(n,random_state=rnd_st)
    concepts.extend(samp['concept'])
    deps.extend([choice(i) for i in samp['deps']])
    
len(deps) == len(concepts)

True

In [4]:
samples = pd.DataFrame({"concept":concepts,"dependency":deps,"source":["llm"]*n+['wiki']*n+['textbook']*n})
samples = samples.sample(frac=1).reset_index(drop=True)
samples['source'].to_csv('../dat/validation/source_column.csv')
samples.loc[:,['concept','dependency']].to_excel('../dat/validation/deps_to_validate.xlsx')